In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str, "SUPERFICIE": str})

In [255]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])

In [256]:
df_temporal = df[df['CAMPAÑA'] != "22"]

df_temporal = df_temporal.drop(axis = 1, columns = ["CAMPAÑA", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]

to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0"))]
l_encoder = LabelEncoder()
train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])

to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\xavid\AppData\Local\Temp\ipykernel_8944\1802291089.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]
C:\Users\xavid\AppData\Local\Temp\ipykernel_8944\1802291089.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0"))]
C:\Users\xavid\AppData\Local\Temp\ipykernel_8944\1802291089.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])


In [257]:
len(df_encoded)

8526

In [258]:
len(train)

2017

In [259]:
len(to_predict)

6509

In [ ]:
import xgboost as xgb
param2 = {}
param2['booster'] = 'gbtree'
param2["eval_metric"] = "error"
param2['eta'] = 0.3
param2['gamma'] = 0
param2['max_depth'] = 6
param2['min_child_weight']=1
param2['max_delta_step'] = 0
param2['subsample']= 1
param2['colsample_bytree']=1
param2['seed'] = 0
param2['base_score'] = 0.5
param2["colsample_bytree"] = 0.4
param2["learning_rate"] = 0.1
param2["n_estimators"] = 2000

model2 = xgb.XGBClassifier(**param2)
model2.fit(train.drop(columns=['SUPERFICIE']), train['SUPERFICIE'])

In [262]:
pred_class = model2.predict(to_predict)

In [263]:
pred_proba = model2.predict_proba(to_predict)

In [264]:
pred_class2 = pred_class +1

In [286]:
nova_col = ((pred_proba >= 0.45)*1).sum(axis=-1)

In [287]:
pred_class2

array([976, 976, 976, ..., 157, 573, 451], dtype=int64)

In [288]:
print(sum(pred_class2 == 0))
print(sum(nova_col == 1))

0
1084


In [289]:
new_to_train = []
for i, j in zip(pred_class2, nova_col):
    if j == 1:
        new_to_train.append(i-1)

In [290]:
new_df = ((pred_proba >= 0.45)*1).sum(axis=-1)
#new_df = new_df.set_index(to_predict.index)

In [291]:
to_predict['SUPERFICIE'] = new_df

In [309]:
to_add_train = to_predict[to_predict["SUPERFICIE"] == 1]
new_to_predict = to_predict[to_predict["SUPERFICIE"] == 0]

In [293]:
to_predict["SUPERFICIE"].unique()

array([1, 0])

In [294]:
to_add_train["SUPERFICIE"] = new_to_train

C:\Users\xavid\AppData\Local\Temp\ipykernel_8944\331182677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_add_train["SUPERFICIE"] = new_to_train


In [295]:
new_train = pd.concat([train, to_add_train])

In [296]:
new_train["SUPERFICIE"] = l_encoder.inverse_transform(new_train["SUPERFICIE"])

In [297]:
new_train["SUPERFICIE"] = new_train["SUPERFICIE"].astype(float)

In [298]:
len(new_train)+len(new_to_predict)

8526

In [353]:
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 20}
model2 = xgb.XGBRegressor(**params_model)
model2.fit(new_train.drop(columns=['SUPERFICIE']), new_train['SUPERFICIE'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.4, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=20, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [354]:
model2.score(new_train.drop(columns=['SUPERFICIE']), new_train['SUPERFICIE'])

0.8099718468451385

In [355]:
new_to_predict["SUPERFICIE"] = model2.predict(new_to_predict.drop(columns = ["SUPERFICIE"]))

C:\Users\xavid\AppData\Local\Temp\ipykernel_8944\1898123790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_to_predict["SUPERFICIE"] = model2.predict(new_to_predict.drop(columns = ["SUPERFICIE"]))


In [356]:
new_to_predict

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,SUPERFICIE
4,5,3,2,3,2,1,1,14126.000,5.622488
5,6,3,2,3,2,1,1,6054.000,5.993777
6,7,4,2,3,2,1,1,12900.000,5.863808
7,8,5,2,4,2,1,1,5450.000,5.393260
8,9,6,2,4,2,1,1,30720.000,7.317737
...,...,...,...,...,...,...,...,...,...
7530,36,8,2,5,1,1,1,6432.335,2.098226
7535,39,8,2,5,1,1,1,2007.876,1.705978
7536,40,8,2,5,1,1,1,2460.471,1.705978
7537,41,8,2,5,1,1,1,2005.133,1.705978


In [357]:
df_final = pd.concat([new_to_predict, new_train])

In [358]:
df_final[df_final["SUPERFICIE"]<0]

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,SUPERFICIE
1112,901,101,16,7,2,1,2,819.50,-0.098583
1116,902,101,16,7,2,1,2,2011.50,-0.246865
4332,902,101,16,10,2,1,2,400.00,-0.122854
4333,902,101,16,7,2,1,2,2736.18,-0.246865
5391,902,101,16,10,2,1,2,740.00,-0.082474
5392,902,101,16,7,2,1,2,2324.60,-0.246865


In [360]:
df_final

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,SUPERFICIE
4,5,3,2,3,2,1,1,14126.000,5.622488
5,6,3,2,3,2,1,1,6054.000,5.993777
6,7,4,2,3,2,1,1,12900.000,5.863808
7,8,5,2,4,2,1,1,5450.000,5.393260
8,9,6,2,4,2,1,1,30720.000,7.317737
...,...,...,...,...,...,...,...,...,...
6489,1094,7,2,9,2,1,2,9490.000,1.380000
6522,40,8,2,5,1,1,1,1329.354,1.660000
7496,1094,7,2,9,2,1,2,6500.000,1.380000
7521,1012,8,3,2,1,1,1,1101.435,0.860000
